In [1]:
#code inspired from https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init

import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils


from torch.autograd import Variable
## each image is 490 x 326 

#create the model classes

#how to initialize weights function
def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

#this is just a layer where you pass in a lambda and it does the rest
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd

    def forward(self, x):
        return self.lambd(x)

#basic block with shortcut (not a bottleneck block)
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='A'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            if option == 'A':
                top = (int) ((self.expansion*planes - in_planes) / 2)
                bot = (self.expansion*planes - in_planes) - top
                self.shortcut = LambdaLayer(lambda x:
                                            F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, top, bot), "constant", 0))
            elif option == 'B':
                self.shortcut = nn.Sequential(
                     nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                     nn.BatchNorm2d(self.expansion * planes)
                )
      

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out
    
##bottleneck block
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1, option='B'):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            if option == 'A':
                top = (int) ((self.expansion*planes - in_planes) / 2)
                bot = (self.expansion*planes - in_planes) - top
                self.shortcut = LambdaLayer(lambda x:
                                                F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, top, bot), "constant", 0)) 
            elif option == 'B':
                    self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion*planes)
                    )
            

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

    
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=2, option='B'):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, option=option)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=4, option=option)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=4, option=option)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, option=option)
        
        self.linear = nn.Linear(4096, num_classes)
        self._initialize_weights()

    def _make_layer(self, block, planes, num_blocks, stride, option):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, option))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

        

In [2]:
def ResNet50():
    return ResNet(Bottleneck, [3,4,6,3])

def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])

In [3]:
import load_data as ld
from load_data import PhoneDataset
from torch.utils.data import DataLoader

train_data = ld.get_data(labels_file='./data/labels/labels.txt', 
                           root_dir='./data/train/');

validation_data = ld.get_data(labels_file='./data/labels/labels.txt',
                                   root_dir='./data/validation/')

test_data = ld.get_data(labels_file='./data/labels/labels.txt',
                       root_dir='./data/train')

train_loader = DataLoader(dataset=train_data, batch_size=2, shuffle=False);
val_loader = DataLoader(dataset=validation_data, batch_size=2)

In [4]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

net = ResNet18().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

for epoch in range(100):
    running_loss = 0.0
    
    for i, (inputs, labels) in enumerate(train_loader, 0):
        net.train()
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = net(inputs.float())
        loss = criterion(outputs.float(), labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        if i % 2 == 1: #print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
            (epoch + 1, i + 1, running_loss / 2))
            running_loss = 0.0
    
    print("Validation loss: ")
    loss = 0.0
    
    for i, (inputs, labels) in enumerate(val_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)
        net.eval()
        with torch.no_grad():
            outputs = net(inputs.float())
            loss += criterion(outputs, labels.float()).item()
    
    print("Epoch Loss: {}".format(loss))
    print("------------------------------------------------------------")
    
print('Finished Training')

[1,     2] loss: 18.961
[1,     4] loss: 7.952
[1,     6] loss: 3.342
[1,     8] loss: 4.382
[1,    10] loss: 6.974
[1,    12] loss: 3.666
[1,    14] loss: 2.789
[1,    16] loss: 12.602
[1,    18] loss: 4.953
[1,    20] loss: 3.610
[1,    22] loss: 1.938
[1,    24] loss: 15.385
[1,    26] loss: 1.815
[1,    28] loss: 0.692
[1,    30] loss: 1.409
[1,    32] loss: 0.677
[1,    34] loss: 0.780
[1,    36] loss: 1.414
[1,    38] loss: 0.460
[1,    40] loss: 0.698
[1,    42] loss: 1.073
[1,    44] loss: 0.765
[1,    46] loss: 1.105
[1,    48] loss: 0.401
[1,    50] loss: 0.481
[1,    52] loss: 0.620
Validation loss: 
Epoch Loss: 116.1207529604435
------------------------------------------------------------
[2,     2] loss: 1.014
[2,     4] loss: 0.303
[2,     6] loss: 0.195
[2,     8] loss: 0.065
[2,    10] loss: 0.704
[2,    12] loss: 0.321
[2,    14] loss: 0.670
[2,    16] loss: 0.737
[2,    18] loss: 0.151
[2,    20] loss: 0.202
[2,    22] loss: 0.316
[2,    24] loss: 0.701
[2,    26] los

[12,    32] loss: 0.046
[12,    34] loss: 0.019
[12,    36] loss: 0.065
[12,    38] loss: 0.086
[12,    40] loss: 0.083
[12,    42] loss: 0.013
[12,    44] loss: 0.049
[12,    46] loss: 0.088
[12,    48] loss: 0.186
[12,    50] loss: 0.156
[12,    52] loss: 0.021
Validation loss: 
Epoch Loss: 0.4504149928689003
------------------------------------------------------------
[13,     2] loss: 0.068
[13,     4] loss: 0.108
[13,     6] loss: 0.098
[13,     8] loss: 0.059
[13,    10] loss: 0.027
[13,    12] loss: 0.091
[13,    14] loss: 0.081
[13,    16] loss: 0.051
[13,    18] loss: 0.033
[13,    20] loss: 0.047
[13,    22] loss: 0.114
[13,    24] loss: 0.066
[13,    26] loss: 0.050
[13,    28] loss: 0.062
[13,    30] loss: 0.082
[13,    32] loss: 0.089
[13,    34] loss: 0.060
[13,    36] loss: 0.005
[13,    38] loss: 0.033
[13,    40] loss: 0.110
[13,    42] loss: 0.097
[13,    44] loss: 0.078
[13,    46] loss: 0.055
[13,    48] loss: 0.099
[13,    50] loss: 0.156
[13,    52] loss: 0.094
Va

[23,    42] loss: 0.019
[23,    44] loss: 0.043
[23,    46] loss: 0.047
[23,    48] loss: 0.042
[23,    50] loss: 0.023
[23,    52] loss: 0.006
Validation loss: 
Epoch Loss: 0.4209532090462744
------------------------------------------------------------
[24,     2] loss: 0.010
[24,     4] loss: 0.020
[24,     6] loss: 0.064
[24,     8] loss: 0.073
[24,    10] loss: 0.050
[24,    12] loss: 0.033
[24,    14] loss: 0.014
[24,    16] loss: 0.007
[24,    18] loss: 0.043
[24,    20] loss: 0.041
[24,    22] loss: 0.078
[24,    24] loss: 0.043
[24,    26] loss: 0.043
[24,    28] loss: 0.017
[24,    30] loss: 0.063
[24,    32] loss: 0.051
[24,    34] loss: 0.068
[24,    36] loss: 0.086
[24,    38] loss: 0.050
[24,    40] loss: 0.022
[24,    42] loss: 0.011
[24,    44] loss: 0.021
[24,    46] loss: 0.041
[24,    48] loss: 0.031
[24,    50] loss: 0.056
[24,    52] loss: 0.052
Validation loss: 
Epoch Loss: 0.551681773737073
------------------------------------------------------------
[25,     2] l

[34,    52] loss: 0.005
Validation loss: 
Epoch Loss: 0.17666017217561603
------------------------------------------------------------
[35,     2] loss: 0.005
[35,     4] loss: 0.011
[35,     6] loss: 0.018
[35,     8] loss: 0.041
[35,    10] loss: 0.015
[35,    12] loss: 0.008
[35,    14] loss: 0.002
[35,    16] loss: 0.004
[35,    18] loss: 0.011
[35,    20] loss: 0.009
[35,    22] loss: 0.006
[35,    24] loss: 0.006
[35,    26] loss: 0.003
[35,    28] loss: 0.004
[35,    30] loss: 0.005
[35,    32] loss: 0.016
[35,    34] loss: 0.020
[35,    36] loss: 0.016
[35,    38] loss: 0.003
[35,    40] loss: 0.006
[35,    42] loss: 0.002
[35,    44] loss: 0.010
[35,    46] loss: 0.011
[35,    48] loss: 0.033
[35,    50] loss: 0.022
[35,    52] loss: 0.012
Validation loss: 
Epoch Loss: 0.20067461393773556
------------------------------------------------------------
[36,     2] loss: 0.009
[36,     4] loss: 0.004
[36,     6] loss: 0.003
[36,     8] loss: 0.005
[36,    10] loss: 0.009
[36,    12

KeyboardInterrupt: 

In [ ]:
  for i, (inputs, labels) in enumerate(val_loader, 0):
    inputs, labels = inputs.to(device), labels.to(device)
    net.eval()
    with torch.no_grad():
        outputs = net(inputs.float())
        print("Net output:")
        print(outputs)
        print("Actual output:")
        print(labels.float())

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [6]:
!nvidia-smi

Sat Jan 18 22:14:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1060    Off  | 00000000:01:00.0  On |                  N/A |
| N/A   52C    P8     8W /  N/A |   2812MiB /  3011MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
torch.cuda.empty_cache()

In [2]:
import torch

x = torch.Tensor(2, 1, 3)
print(x.size())
print(x.squeeze().size())

torch.Size([2, 1, 3])
torch.Size([2, 3])


In [5]:
!nvidia-smi

Sat Jan 18 23:18:31 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1060    Off  | 00000000:01:00.0  On |                  N/A |
| N/A   66C    P2    34W /  N/A |   2972MiB /  3011MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  